<a href="https://colab.research.google.com/github/sumitwaghmare/bootstrap-material-design/blob/master/Prophet_ts_mysql.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!pip install mysql-connector
!pip install scikit-learn scipy

In [ ]:
import pandas as pd
from pandas import DataFrame
from fbprophet import Prophet
import mysql.connector
from fbprophet.plot import add_changepoints_to_plot
from fbprophet.diagnostics import performance_metrics
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

import smtplib

server = smtplib.SMTP('smtp.gmail.com', 587)
server.starttls()
server.login("papasumit@gmail.com", "isvmnhomehsqhcqj")

mydb = mysql.connector.connect(
  host="dbgoodorder.coliq5gf8jmk.ap-south-1.rds.amazonaws.com",
  user="admin",
  password="J72hsAZNQuQs",
  database = "newschema"
)
#print(mydb)
curr1 = mydb.cursor()

curr1.execute("select DISTINCT productSKU, storeID from saleshistory")

result1 = curr1.fetchall()

for x in result1:
  curr2 = mydb.cursor()
  curr2.execute("select * from saleshistory where productSKU='" + x[0] + "' and storeID ='"+x[1]+"' and status = 'actual' order by dateInfo")
  result2 = curr2.fetchall()
  print(result2)
  field_names = [i[0] for i in curr2.description]
  #print(result2)
  #print(curr2.rowcount, "record found.")
  df = DataFrame(result2)
  df.columns = field_names
  df['ds'] = pd.to_datetime(df['dateInfo'])
  df['y'] = df['qty']
  #print(df)
  #print(x[0],x[1])
  prophet = Prophet()
  prophet.fit(df) 
  #future = prophet.make_future_dataframe(periods=365, freq='D')
  #forecast = prophet.predict(future)
  future = prophet.make_future_dataframe(periods=365, include_history=True)
  forecast = prophet.predict(future)
  metric_df = forecast.set_index('ds')[['yhat']].join(df.set_index('ds').y).reset_index()
  metric_df.dropna(inplace=True)
  #print(metric_df)
  print(mean_absolute_error(metric_df.y, metric_df.yhat))
  #save MAD/MAE same
  curr4 = mydb.cursor()
  curr4.execute(f'UPDATE inventoryDB SET mad = {mean_absolute_error(metric_df.y,metric_df.yhat)} WHERE productSKU = "{x[0]}" and storeID = "{x[1]}"')
  mydb.commit()
  curr4.close()
  for ind in forecast.index:
    print(forecast['ds'][ind], forecast['yhat'][ind])   
    curr3 = mydb.cursor()
    sql = f'INSERT IGNORE INTO saleshistory (dateInfo, productSKU,storeID,qty,clientCode,status,CustomerID) VALUES ("{forecast["ds"][ind]}","{x[0]}","{x[1]}","{forecast["yhat"][ind]}","CL01","forecast","PE")'
    #print(sql)
    curr3.execute(sql)
    mydb.commit()
    print(curr3.rowcount, "Record inserted successfully into table")
    curr3.close()
  
  #fig = prophet.plot(forecast,xlabel='Date', ylabel=x[0])
  #ax = fig.gca()
  #ax.set_title("Title" + x[0] + ","+x[1], size=34)
  #a = add_changepoints_to_plot(fig.gca(), prophet, forecast)
  #print(future)

msg = "COLAB WORK FINISH ALERT!"
server.sendmail("papasumit@gmail.com", "papasumit@gmail.com", msg)
server.quit()

In [ ]:
df

In [ ]:
forecast

In [ ]:
metric_df = forecast.set_index('ds')[['yhat']].join(df.set_index('ds').y).reset_index()
metric_df
metric_df.dropna(inplace=True)
metric_df
mean_absolute_error(metric_df.y, metric_df.yhat)

PO Calculation using Python

---


logic as follows

In [ ]:
#PO Calculation
#print 15 forecast first
